In [1]:
import pandas as pd

In [2]:

# read in the datasets
X_test = pd.read_csv("../data/multiple_turbine_data/X_test.csv")
y_test = pd.read_csv("../data/multiple_turbine_data/y_test.csv")
X_train = pd.read_csv("../data/multiple_turbine_data/X_train.csv")
y_train = pd.read_csv("../data/multiple_turbine_data/y_train.csv")

In [3]:
# combine the datasets
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])
# drop the # Date and Time columns
y = y.drop(['# Date and time'], axis=1)

all_data = pd.concat([X, y], axis=1)

In [4]:
all_data.columns = all_data.columns.str.strip()  # Strip any leading or trailing whitespace

# Melt the dataframe without creating a Turbine column first
melted_df = all_data.melt(id_vars=['# Date and time'], var_name='Indicator', value_name='Value')

# Extract the turbine number from the 'Indicator' column
melted_df['Turbine'] = melted_df['Indicator'].str.extract(r'(\d+)_')

# Clean the 'Indicator' column by removing the turbine number prefix
melted_df['Indicator'] = melted_df['Indicator'].str.replace(r'^\d+_', '', regex=True)

# Display the transformed dataframe
melted_df = melted_df[['# Date and time', 'Turbine', 'Indicator', 'Value']]

# rename '# Date and time' to 'DateTime'
melted_df.rename(columns={'# Date and time': 'DateTime'}, inplace=True)
melted_df.head()


,DateTime,Turbine,Indicator,Value
0,2016-05-03 09:40:00,1,Wind speed (m/s),4.728757
1,2016-05-03 09:50:00,1,Wind speed (m/s),6.427782
2,2016-05-03 10:00:00,1,Wind speed (m/s),5.876329
3,2016-05-03 10:10:00,1,Wind speed (m/s),5.676786
4,2016-05-03 10:20:00,1,Wind speed (m/s),4.777902


In [12]:
# change Turbine to int
melted_df['Turbine'] = melted_df['Turbine'].astype(int)

# change Turbine from 1,2 etc to T01, T02 etc
melted_df['Turbine'] = 'T' + melted_df['Turbine'].apply(lambda x: f'{x:02d}')

In [15]:
# drop index
melted_df.reset_index(drop=True, inplace=True)
melted_df.to_parquet("../data/multiple_turbine_data/all_data.parquet", index=False)